### Classes & Metaclasses
From a great talk by James Powell

Link: https://www.youtube.com/watch?v=7lmCu8wz8ro


In [1]:
class Polynomial:
    def __init__(self, *coeffs):
        self.coeffs = coeffs
        
    # repr = represent 
    # the *{!r} is the repr ... 
    def __repr__(self):
        return 'Polynomial(*{!r})'.format(self.coeffs)
    
    # zip() takes an iterable and outputs an iterable... 
    def __add__(self, other):
        return Polynomial(*(x + y for x, y in zip(self.coeffs, other.coeffs)))
    
    # you have to look at the documentation... len is pre-defined in some sense
    # len is thematically in the documentation, to evoke some sense of size
    # the pattern is to think of the higher level function acting on self... hence len(self.coeffs) 
    def __len__(self):
        return len(self.coeffs)
    
    def __call__(self):
        pass

In [2]:
p1 = Polynomial(1, 2, 3)
p2 = Polynomial(3, 4, 3)

p1 + p2

Polynomial(*(4, 6, 6))

In [3]:
len(p1) 

3

### Metaclasses 



In [4]:
# File 1 - library.py
# This is the infrastructure side... 
class Base:
    def food(self):
        return 'foo'

In [13]:
# File2 - user.py
# This is the user developer side... assert that this exists in the library side before you execute. 
# This way, you can catch the error before you go into production. 
assert hasattr(Base, 'foo'), "you broke it, you fool!"

class Derived(Base):
    def bar(self):
        return self.foo

In [6]:
# File 1 - library.py
# now the shoe is on the other foot... 
# and now the library side is using somethin that's made on the user side??? not sure (RETURN TO THIS)


class Base:
    def foo(self):
        return self.bar()
    

In [7]:
# File2 - user.py

assert hasattr(Base, 'foo'), "you broke it, you fool!"

class Derived(Base):
    def bar(self):
        return 'bar'

In [9]:
Derived.bar

<function __main__.Derived.bar>

In [10]:
def _():
    class Base:
        pass

from dis import dis

In [11]:
# dis = dissembles

dis(_) # LOAD_BUILD_CLASS

  2           0 LOAD_BUILD_CLASS
              2 LOAD_CONST               1 (<code object Base at 0x1064e39c0, file "<ipython-input-10-f0669648f7a8>", line 2>)
              4 LOAD_CONST               2 ('Base')
              6 MAKE_FUNCTION            0
              8 LOAD_CONST               2 ('Base')
             10 CALL_FUNCTION            2
             12 STORE_FAST               0 (Base)
             14 LOAD_CONST               0 (None)
             16 RETURN_VALUE


## Great introduction to decorators
Core idea is to think about wrappers. If you have a function that returns a function... There is an important concept here as a closure object duality but we won't go into this. 

- Fundamentally, you can wrap wide swathes of functions without having to re-write a lot of user code or have a lot of churn in the library code. 
- `*args and  **kwargs` --> can take arbitrary parameter specs and forward it along in any fashion. `*args` is used to send a non-keyworded variable length argument to the function. `**kwargs` allows you to pass keyworded variable length of arguments to a function. You should use `**kwargs` if you want to handle named arguments in a function. 


In [2]:
from time import time

# you can define functions anywhere, at run-time too
def timer(func): 
    def f(*args, **kwargs): 
        before = time() # wrap the function with behavior before and after
        rv = func(*args, **kwargs) # rv = return value
        after = time() 
        print('elapsed', after - before) 
        return rv
    return f

@timer
def add(x, y=10): 
    return x + y
# the decorator replaces the need for this line below
# a decorator is syntax that's equal to function = wrapper(function)
# you can add in this functionality without needing to re-write this 
# extra user code...! now you can write it at the top.
# add = timer(add)

@timer
def sub(x, y=10): 
    return x - y
# sub = timer(sub)

print('add(10)', add(10))
print('add(20,30)', add(20,30))

elapsed 9.5367431640625e-07
add(10) 20
elapsed 9.5367431640625e-07
add(20,30) 50


### Generators
This is more than just laziness vs eagerness. Eagerness refers to this idea that irrespective of what we care about this function always takes the exact same amount of memory and the exact same amount of time. If we care only about the first element, it takes the same amount of memory and time as if we cared about the entire list of numbers. It `eagerly` gives the entire result and we're `waiting` for the entire result. 

- Generators inter-weave them... the idea of co-routines. 
- Sub-routines are pieces of executable code that run from a single entry point and one single exit point. They run and they're done. For library code, the sub-routine runs and then finishes in library code, and then the user code needs to pick it up. 
- For the generator, and the co-routine, as you ask for values the generator runs... some user code runs, generator runs and yields to the user code, the user code goes back and does its work, then goes back to the generator and asks for another value and so on. 

In [4]:
'''
functionally, what is the difference here? --> probably none
but what if you wanted to add state-ful behavior, then clearly
there are different ways to do it for the class or the function
'''

def add(x,y): 
    return x + y

class Adder: 
    
    def __init__(self): 
        self.z = 0 
    
    def __call__(self, x, y):
        return x + y + self.z 

add2 = Adder() 

In [7]:
from time import sleep 

def add1(x,y): 
    return x + y

class Adder:
    def __call__(self,x,y): 
        return x + y
add2 = Adder() 

# Let's try to mimic a function that takes a while to get
# information sent back to it, like we're pinging a network
def compute(): 
    rv = []
    for i in range(10): 
        sleep(.5) # so 10*0.5 = 5 seconds to run through
        rv.append(i) 
    return rv

In [8]:
compute() # takes 5 seconds

# what if this is 1m entries? maybe you want to look at them 1 by 1!
# and maybe if we can look at a couple of them and then throw it out
# if it doesn't fit our parameters. 

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [16]:
# let's rewrite compute as a class

from time import sleep 

def add1(x,y): 
    return x + y

class Adder:
    def __call__(self,x,y): 
        return x + y
add2 = Adder() 

# nothing really changed... 
class Compute: 
    def __call__(self): 
        rv = []
        for i in range(10): 
            sleep(5) 
            rv.append(i) 
        return rv
    
    def __iter__(self): 
        self.last = 0
        return self

    # looks at the last value we looked at 
    # sleeps.. then increments it by 1
    # if we've looked at too many it stops the iteration
    
    # disclosure: this is a pain, AND generators are much cleaner!
    def __next__(self): 
        rv = self.last
        self.last += 1
        if self.last > 10: 
            raise StopIteration()
        sleep(0.5)
        return rv
    
#for val in Compute():
#    print(val)
    
# for x in xs: 
#    pass
    
# xi = iter(xs)    --> __iter__
# while True: 
#    x = next(xi) --> __next__

In [15]:
# this is very ugly ... so we use generators! 
# generator formulation!! 

def compute(): 
    for i in range(10): 
        sleep(0.5)
        yield i
        
# for val in compute(): 
#     print(val)

### Idea of sequencing & APIs
Generators... it's more than a question of eagerness or laziness. The mental model at work here is that generators are a mechanism by which you can create code that can interweave with other code and enforce sequencing. The generator FORCES the sequencing on someone. The generator is a co-routine that interweaves user code and library code. 

In [13]:
# Api intends us to INTERWEAVE CODE.
# that's why don't just clump up all the code into one. 

class Api: 
    def run_this_first(self): 
        first()
    def run_this_second(self): 
        second()
    def run_this_last(self): 
        last()
        
        
class api(): 
    first()
    yield
    second()
    yield
    last() 


### Context Manager
Common metaphor here... in C++ it's resource allocation initialization. It's this idea that you want to do some setup and teardown and you want to combine them. 